<a href="https://colab.research.google.com/github/amitsiwach/CNN/blob/main/DenseNet/DenseNet169_on_Flower_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from glob import glob
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
# from keras.applications.vgg16 import preprocess_input
from keras.applications.densenet import preprocess_input

In [3]:
input_shape_densenet = (224, 224, 3)

densenet_model = keras.applications.DenseNet169(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape_densenet,
    pooling=None
)

In [4]:
# don't train existing weights
for layer in densenet_model.layers:
  layer.trainable = False

In [5]:
# useful for getting number of classes
folders = glob('/content/drive/MyDrive/Datasets/data for VGG Net/train/*')
print(len(folders))

4


In [6]:
#Give dataset path
train_path = '/content/drive/MyDrive/Datasets/data for VGG Net/train'
test_path = '/content/drive/MyDrive/Datasets/data for VGG Net/test'

In [7]:
x = Flatten()(densenet_model.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=densenet_model.input, outputs=prediction)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

In [8]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [9]:
# Data Augmentation
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [10]:
# Make sure you provide the same target size as initialied for the image size
train_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/data for VGG Net/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 475 images belonging to 4 classes.


In [11]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/data for VGG Net/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 264 images belonging to 4 classes.


In [12]:
from keras import optimizers


sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [13]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

#lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

#num_epochs = 1000
#num_batch_size = 32

checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
                               verbose=1, save_best_only=True)

callbacks = [checkpoint, lr_reducer]


model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=5,
  validation_steps=32,
    callbacks=callbacks ,verbose=1)

<ipython-input-13-3b8e16bac612>:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10
5/5 [==============================] - ETA: 0s - loss: 11.1359 - accuracy: 0.6323


Epoch 1: val_loss improved from inf to 5.82331, saving model to mymodel.h5
5/5 [==============================] - 106s 22s/step - loss: 11.1359 - accuracy: 0.6323 - val_loss: 5.8233 - val_accuracy: 0.8447 - lr: 0.0100
Epoch 2/10
5/5 [==============================] - ETA: 0s - loss: 4.1141 - accuracy: 0.8938

5/5 [==============================] - 34s 7s/step - loss: 4.1141 - accuracy: 0.8938 - lr: 0.0100
Epoch 3/10
5/5 [==============================] - ETA: 0s - loss: 1.1026 - accuracy: 0.9548

5/5 [==============================] - 38s 6s/step - loss: 1.1026 - accuracy: 0.9548 - lr: 0.0100
Epoch 4/10
5/5 [==============================] - ETA: 0s - loss: 2.8245 - accuracy: 0.9625

...

5/5 [==============================] - 35s 7s/step - loss: 1.7488 - accuracy: 0.9812 - lr: 0.0100
Epoch 7/10
5/5 [==============================] - ETA: 0s - loss: 0.6689 - accuracy: 0.9937

5/5 [==============================] - 34s 7s/step - loss: 0.6689 - accuracy: 0.9937 - lr: 0.0100
Epoch 8/10
5/5 [==============================] - ETA: 0s - loss: 5.3269 - accuracy: 0.9484

5/5 [==============================] - 34s 6s/step - loss: 5.3269 - accuracy: 0.9484 - lr: 0.0100
Epoch 9/10
5/5 [==============================] - ETA: 0s - loss: 1.4731 - accuracy: 0.9935

5/5 [==============================] - 33s 7s/step - loss: 1.4731 - accuracy: 0.9935 - lr: 0.0100
Epoch 10/10
5/5 [==============================] - ETA: 0s - loss: 1.2700 - accuracy: 0.9935

5/5 [==============================] - 33s 6s/step - loss: 1.2700 - accuracy: 0.9935 - lr: 0.0100
